In [0]:
!git clone https://github.com/osafty/ColorDeblurring.git

In [0]:
import os

In [0]:
os.chdir('/content/ColorDeblurring/first implementation')

In [0]:
!pip install keras==2.2.5 tensorflow==1.14 pillow h5py jupyter scikit-image
import keras
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import random
import tensorflow as tf

In [0]:
#to delete a folder with all its files
#import shutil
#shutil.rmtree('/path/to/directory/Test')

In [0]:
X = []
for filename in os.listdir('./Train/'):
    imag = load_img('./Train/'+filename)
    if imag.size[1] > imag.size[0]:
      imag = imag.rotate(90, expand = True)
    imag = imag.crop(box = [0,0,imag.size[1],imag.size[1]]) #cropped to have square image
    X.append(img_to_array(imag.resize((256,256)))) 
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

In [0]:
#complex architecture
model = Sequential()
#model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))



In [0]:
# simple architecture
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [0]:
from keras.callbacks import ModelCheckpoint

In [0]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 1
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)


checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)
#opt = keras.optimizers.Adam(lr=0.1)
opt = keras.optimizers.rmsprop(lr=0.001)
model.compile(loss='mse', optimizer=opt)      
#model.compile(optimizer='rmsprop', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), epochs=20, steps_per_epoch=1)#, callbacks=[checkpoint])
#model.fit(x=X, y=Y, batch_size=10, epochs=1000)
#model.fit_generator(image_a_b_gen(batch_size,X,Y), epochs=1000, steps_per_epoch=1, callbacks=[checkpoint])


In [0]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [0]:
os.mkdir('result')

In [15]:
color_me = []
for filename in os.listdir('./Test/'):
    imag = load_img('./Test/'+filename)
    if imag.size[1] > imag.size[0]:
      imag = imag.rotate(90, expand = True)
    imag = imag.crop(box = [0,0,imag.size[1],imag.size[1]]) #cropped to have square image
    color_me.append(img_to_array(imag.resize((256,256))))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256,256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("./result/img_"+str(i)+".png", lab2rgb(cur))

/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 5 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [18]:
!zip -r ./result.zip ./result
from google.colab import files
files.download("./result.zip")

  adding: result/ (stored 0%)
  adding: result/img_0.png (deflated 0%)
